# Практическое задание к уроку 3. Логистическая регрессия. Log Loss

In [33]:
import numpy as np
import matplotlib.pyplot as plt

In [34]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [49]:
def calc_logloss(y, y_pred,step=1e-4):
    y_pred[y_pred==0]=step
    y_pred[y_pred==1]=1-step
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [50]:
step=1e-4
y1 = np.array([1, 0])
y_pred1 = np.array([1.0, 0.0])

calc_logloss(y1,y_pred1,1e-8)

1.0000000100247594e-08

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [51]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [89]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
       # if i % (iterations / 10) == 0:
       #     print(i, W, err)
    return W

In [157]:
def optimise(iteretion,alfa):
    a_error=[]
    for i in iteretion:
        for j in alfa:
            W=eval_model(X, y, i, alpha=j)
            z = np.dot(W, X)
            y_pred = sigmoid(z)
            err = calc_logloss(y, y_pred)
            a_error.append([err,i,j])
    a_error.sort()
    print(f'err = {a_error[0][0]} iteretion = {a_error[0][1]} alfa = {a_error[0][2]}')

In [158]:
iteretion=[1000,10000,100]
alfa=[1e-3,1e-4,1e-2]
err=optimise(iteretion,alfa)

err = 4.605220188488314 iteretion = 1000 alfa = 0.001


### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [54]:
def calc_pred_proba(W,X): 
    p = 1/(1 + np.e**(-np.dot(W,X)))
    return p

In [55]:
W=eval_model(X, y, 1000, alpha=1e-4)

100 [ 0.49570084 -0.14048527  0.04121298  1.52293455] 2.709936347663394
200 [ 0.49526142 -0.14104576  0.03799644  1.52386752] 1.5738008657051465
300 [ 0.49482342 -0.14160149  0.03701973  1.52480441] 1.4565078128769913
400 [ 0.49438611 -0.14216368  0.03302997  1.52573682] 1.1472639434455152
500 [ 0.49396513 -0.14270117  0.041088    1.52669295] 2.4784460715959393
600 [ 0.4935256  -0.1432587   0.03886659  1.52762721] 4.806729135570135
700 [ 0.49308873 -0.14381081  0.0395528   1.52856698] 1.8413768921908893
800 [ 0.49265161 -0.14436458  0.03937641  1.52950528] 4.504609460153813
900 [ 0.49221778 -0.14491722  0.03972581  1.53044536] 1.881912987193233
1000 [ 0.49177227 -0.14548424  0.03306915  1.53137102] 1.14986814973062


In [56]:
calc_pred_proba(W,X)

array([0.99999999, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99999995, 1.        ])

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [57]:
def calc_pred(W,X): 
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    return y_pred

In [58]:
y_pred=calc_pred(W,X)
y_pred

array([0.99999999, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99999995, 1.        ])

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [59]:
accuracy=np.sum(y==y_pred)/len(y)
accuracy

0.2

In [60]:
def get_confusion_matrix(y_true, y_pred):
    confusion=[]
    cl_1=[0,0]
    cl_0=[0,0]
    for i in range(0,y_true.shape[0]):
        if y_true[i]==round(y_pred[i],0):
            if y_true[i]==1:
                cl_1[0]+=1
                # TP
            else:
               # TN
                cl_1[1]+=1    
        if y_true[i]!=round(y_pred[i],0):  
            if y_true[i]==1:
                # FP
                cl_0[0]+=1
            else:
               # FN
                cl_0[1]+=1
    confusion.append(cl_1)
    confusion.append(cl_0)
    return confusion        

In [61]:
confusion_matrix=get_confusion_matrix(y, y_pred)
confusion_matrix

[[5, 0], [0, 5]]

Точность (precision) 
$$precision(a, X) = \frac{TP}{TP+FP}.$$

In [62]:
precision=confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[0][1])
precision

1.0

Полнота (recall) 

$$recall(a, X) = \frac{TP}{TP+FN},$$


In [63]:
recall=confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][1])
recall                              

0.5

$$F_{1} = {2\cdot} \frac{precision \cdot recall}{precision + recall}.$$

In [68]:
F_1=2*precision*recall/(precision+recall)
F_1

0.6666666666666666

### 6. Могла ли модель переобучиться? Почему?

W= 0.48796522, -0.13417284, -0.00244908,  1.5266554 отсутствует регуляризация весов и масштабирование признаков точность =1   полноты 0,5  по сравнению данных модель переобучена 

### 7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [64]:
def eval_model_l1(X, y, iterations, alpha=1e-4,lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)+ lambda_*np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [65]:
def eval_model_l2(X, y, iterations, alpha=1e-4,lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)+ lambda_*W)/2
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W